## Make and view annual time-series data as 3-band composite

In [ ]:
import os
import sys
from pathlib import Path
import datetime

import rasterio
from rasterio import plot
from rasterio.plot import show
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
sys.path.append(r"../LUCinSA_helpers")
from TScomposite import *

In [ ]:
'''
PARAMETERS: modify basicConfig in Notebook_settings notebook, then run that notebook and this cell to update here
DO not modify this cell
'''

%store -r basicConfig
print("Basic Parameters: \n PrintDate = {} \n gridCell = {} \n index_dir = {} \n out_dir = {} \n spec_index= {} \n plotYr = {}"
      .format(basicConfig['today'],basicConfig['gridCell'],basicConfig['index_dir'],basicConfig['out_dir'],basicConfig['spec_index'], basicConfig['plotYr']))

#%store -r TSCompositeParams
#print("TSComposite Parameters: \n BandsOut = {}".format(TSCompositeParams['BandsOut']))

In [ ]:
#Basic plotting scripts

def normalize(array):
    array_min, array_max = array.min(), array.max()
    return (array - array_min) / (array_max - array_min)

def plotrgb(raster):
    # Convert to numpy arrays
    red = raster.read(3)
    green = raster.read(2)
    blue = raster.read(1)

    # Normalize band DN
    red_norm = normalize(red)
    green_norm = normalize(green)
    blue_norm = normalize(blue)

    # Stack bands
    rgb = np.dstack((red_norm, green_norm, blue_norm))

    return rgb

## Current options for composite bands are:
* Avg (mean)
* Max (maximum index value)
* Min (minumum index value)
* Amp (Max - Min)
* Std (standard deviation)
* CV (coefficient of variation: std/mean)
* MaxDate (day of year on which maximum occurs)
* MaxDateCos (cosine of max date, to reduce periodic effect)
* MinDate (day of year on which minimum occurs)
* MinDateCos (cosine of min date, to reduce periodic effect)
---------------------------------------------------------------
* Jan (value from Jan 10th image)
* Apr (value from Apr 10th image)
* Jun (value from Jun 10th image)
* Aug (value from Aug 10th image)
* Nov (value from Nov 10th image)

### note: if running many these, use bash script "MakeTScomposite.sh" to run on job node of cluster

## Make ['Min','Max','Amp'] composite to highlight crops vs grassland/forest

In [ ]:
##Note: this is a tmp local check before moving to cluster:
local_in_dir = "C:/Users/klobw/Desktop/NasaProject/Paraguay/Grids/000993/evi2_2yr"
local_out_dir = "C:/Users/klobw/Desktop/Testing"
Comp1 = MakeTSComposite(basicConfig['gridCell'], local_in_dir,local_out_dir,basicConfig['plotYr'],basicConfig['spec_index'],['Min','Max','Amp'])

#Comp1 = MakeTSComposite(basicConfig['gridCell'], basicConfig['index_dir'],basicConfig['out_dir'],basicConfig['plotYr'],basicConfig['spec_index'],['Min','Max','Amp'])

In [ ]:
raster = rasterio.open(os.path.join(local_out_dir,'{:06d}'.format(basicConfig['gridCell'])+'_'+str(basicConfig['plotYr'])+basicConfig['spec_index']+'_'+'MinMaxAmp.tif'))
#raster = rasterio.open(os.path.join(basicConfig['out_dir'],'{:06d}'.format(basicConfig['gridCell'])+'_'+str(basicConfig['plotYr'])+basicConfig['spec_index']+'_'+'MinMaxAmp.tif'))
rgb = plotrgb(raster)
fig = plt.figure(figsize=(18,12))
plt.imshow(rgb);

## Make ['Jan','Jun','Nov'] composite to see different crops

In [ ]:
Comp2 = MakeTSComposite(basicConfig['gridCell'], local_in_dir,local_out_dir,basicConfig['plotYr'],basicConfig['spec_index'],['Jan','Jun','Nov'])
#Comp2 = MakeTSComposite(basicConfig['gridCell'], basicConfig['index_dir'],basicConfig['out_dir'],basicConfig['plotYr'],basicConfig['spec_index'],['Jan','Jun','Nov'])

In [ ]:
raster = rasterio.open(os.path.join(local_out_dir,'000993_2021evi2_JanJunNov.tif'))
#raster = rasterio.open(os.path.join(basicConfig['out_dir'],'{:06d}'.format(basicConfig['gridCell'])+'_'+str(basicConfig['plotYr'])+basicConfig['spec_index']+'_'+'JanJunNov.tif'))
rgb = plotrgb(raster)
fig = plt.figure(figsize=(18,12))
plt.imshow(rgb);

# To save an html copy of this notebook with all outputs:

In [ ]:
### Run to print output as html

outName = str(basicConfig['country']+'2b_ViewTimeSeriesComposite'+'_cell'+str(basicConfig['gridCell'])+'_'+basicConfig['plotYr'])
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --output=$outName 1b_ExploreData_OpenImage.ipynb